In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
import pandas as pd
import numpy as np
from wavhandler import *

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [2]:
data1 = Dataset('Leafminers')
data1.read(data=data1.target_classes[0], setting='psd_dB', labels='text')

data2 = Dataset('LG')
data2.read(data=data2.target_classes[0], setting='psd_dB', labels='text')

data3 = Dataset('LG')
data3.read(data=data3.target_classes[1], setting='psd_dB', labels='text')

data4 = Dataset('Pcfruit')
data4.read(data=data4.target_classes[1], setting='psd_dB', labels='text')

Data: Cichorii.
Read 1451 filenames in 0.04 seconds.
Loaded data into matrix in 6.12 seconds.
Data: LG_drosophila_10_09.
Read 5536 filenames in 0.08 seconds.
Loaded data into matrix in 21.73 seconds.
Data: LG_zapr_26_09.
Read 7210 filenames in 0.08 seconds.
Loaded data into matrix in 28.13 seconds.
Data: D. suzukii.
Read 2401 filenames in 0.02 seconds.
Loaded data into matrix in 9.16 seconds.


In [3]:
data1.clean(plot=False)
data2.clean(plot=False)
data3.clean(plot=False)
data4.clean(plot=False)

In [8]:
big = pd.DataFrame()

big = pd.concat([data1.X, data2.X, data3.X, data4.X], axis=0)
big['y'] = pd.concat([data1.y, data2.y, data3.y, data4.y], axis=0)
big.dropna(how='any', axis=1, inplace=True)
big.shape

(15405, 130)

In [9]:
big.iloc[:,-1].value_counts()

LG_zapr_26_09          7002
LG_drosophila_10_09    5277
D. suzukii             2373
Cichorii                753
Name: y, dtype: int64

## Train 2D Model

In [7]:
from utils_train import make_classification_conv1d, make_classification_conv2d

big_names = pd.DataFrame(pd.concat([data1.filenames, data2.filenames, 
                                    data3.filenames, data4.filenames], axis=0), columns=['filenames']).reset_index(drop=True)
big_names['y'] = big_names.filenames.apply(lambda x: x.split('/')[6])
#pd.concat([data1.y, data2.y, data3.y, data4.y], axis=0).reset_index(drop=True)

# big_names.dropna(how='any', axis=0, inplace=True)
big_names.sample(5)
big_names.y.value_counts()
make_classification_conv2d(big_names.filenames.tolist(), big_names.y, undersampling=False)

Class balance: 
3    7210
2    5536
1    2401
0    1451
Name: 0, dtype: int64

Epoch 1/100
374/374 [==============================] - 83s 221ms/step - loss: 0.1806 - acc: 0.9321 - val_loss: 1.4274 - val_acc: 0.7091

Epoch 00001: val_acc improved from -inf to 0.70913, saving model to /media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/temp_data/test__stft_wrapper.h5
Epoch 2/100
374/374 [==============================] - 59s 157ms/step - loss: 0.0972 - acc: 0.9620 - val_loss: 1.2891 - val_acc: 0.7609

Epoch 00002: val_acc improved from 0.70913 to 0.76095, saving model to /media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/temp_data/test__stft_wrapper.h5
Epoch 3/100
374/374 [==============================] - 59s 157ms/step - loss: 0.0751 - acc: 0.9723 - val_loss: 1.3968 - val_acc: 0.7734

Epoch 00003: val_acc improved from 0.76095 to 0.77344, saving model to /media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/temp_data/test__stft_wrapper.h5
Epoch 4/100


## Testing trained 2D Model

In [55]:
from utils_train import *
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

path = './temp_data/'
yaml_file = open(path + 'test__stft_wrapper.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights(path + "test__stft_wrapper.h5")
print("Loaded model from disk")

loaded_model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

le = LabelEncoder()
yy = le.fit_transform(big_names.y)

X = big_names.iloc[:,:-1]
y = yy
targets = len(np.unique(y))

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y, random_state=0)

Loaded model from disk


In [80]:
batch_size = 16
loaded_model.evaluate_generator(valid_generator(X_test.filenames.tolist(), 
                                                y_test.tolist(), 
                                                batch_size=batch_size, 
                                                setting='stft', 
                                                target_names=np.unique(y_test).tolist()),
                                steps = int(math.ceil(float(len(X_test)) / float(batch_size))),
                                verbose=1)

104/104 [==============================] - 4s 36ms/step


[0.023080089985949116, 0.9891566265060241]

## Train 1D model

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
yy = le.fit_transform(big.y)

make_classification_conv1d(big.iloc[:,:-1], yy)

Train on 11091 samples, validate on 2773 samples
Epoch 1/100
11091/11091 [==============================] - 8s 707us/step - loss: 0.2838 - acc: 0.8931 - val_loss: 13.6324 - val_acc: 0.1536

Epoch 00001: val_acc improved from -inf to 0.15362, saving model to /media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/temp_data/test_.h5
Epoch 2/100
11091/11091 [==============================] - 4s 318us/step - loss: 0.1871 - acc: 0.9346 - val_loss: 4.5148 - val_acc: 0.2575

Epoch 00002: val_acc improved from 0.15362 to 0.25748, saving model to /media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/temp_data/test_.h5
Epoch 3/100
11091/11091 [==============================] - 4s 336us/step - loss: 0.1649 - acc: 0.9372 - val_loss: 12.7893 - val_acc: 0.1583

Epoch 00003: val_acc did not improve from 0.25748
Epoch 4/100
11091/11091 [==============================] - 4s 322us/step - loss: 0.1473 - acc: 0.9466 - val_loss: 0.8259 - val_acc: 0.7288

Epoch 00004: val_acc improved from

## Testing trained 1D model

In [20]:
from keras.models import model_from_yaml

In [21]:
path = './temp_data/'
yaml_file = open(path + 'test__raw_final.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights(path + "test_.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Loaded model from disk


In [24]:
loaded_model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [31]:
from utils_train import *
from keras.utils import to_categorical

X = big.iloc[:,:-1]
y = yy
targets = len(np.unique(y))

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y, random_state=0)

# Convert label to onehot
y_train = to_categorical(y_train, num_classes=targets)
y_val = to_categorical(y_val, num_classes=targets)
y_test = to_categorical(y_test, num_classes=targets)

X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


loaded_model.evaluate(X_test, y_test, verbose=1)

1541/1541 [==============================] - 3s 2ms/step


[0.06758605744891409, 0.972744970798183]

In [ ]:
data1.get_sensor_features()

In [ ]:
data1.df_features.hist()

In [ ]:
data2.get_sensor_features(version='2')

In [ ]:
data2.df_features.hist()

In [ ]:
dt1 = Dataset('Leafminers')
dt1.read(data=dt1.target_classes[0], setting='read', labels='text')
dt1.get_frequency_peaks()

In [ ]:
dt2 = Dataset('Leafminers')
dt2.read(data=dt1.target_classes[1], setting='read', labels='text')
dt2.get_frequency_peaks()

In [ ]:
perform_analysis(X,y)

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
data2.get_sensor_features(version='2')

In [ ]:
def clean_inds(dataset, th):
    df = dataset.X.copy()
    dataset.y.index = list(dataset.y.reset_index(drop=True).index)
    df['var'] = df.apply(lambda x: x.iloc[10:50].var(), axis=1)
    inds = df[(df['var']>th)].index
    return inds.values

In [ ]:
np_hist(data2.df_features.loc[clean_inds(data2, 10)].dropna(), 'temperature')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,8))
np_hist(data2.df_features, 'date_hour', rot=0, fs=14)


In [ ]:
plt.figure(figsize=(14,8))
np_hist(data2.df_features, 'humidity')

In [ ]:
data2.df_features.date.sort_values(ascending=False)